In [5]:
import pandas as pd
import numpy as np
import os

In [2]:
all_categories = "Arts and Entertainment·Cars & Other Vehicles·Computers and Electronics·Education and Communications·Family Life·Finance and Business·Food and Entertaining·Health·Hobbies and Crafts·Holidays and Traditions·Home and Garden·Personal Care and Style·Pets and Animals·Philosophy and Religion·Relationships·Sports and Fitness·Travel·Work World·Youth"
all_categories = all_categories.lower()
all_categories = all_categories.split("·")

df = pd.read_csv("data/wikihowSep.csv")
df = df[df.sectionLabel != "Steps"]
df["title"] = df["title"].str.lower()
df["sectionLabel"] = df["sectionLabel"].str.lower()

wikiCat = pd.read_csv("data/cate.csv", sep=",", error_bad_lines=False, names=["title", "category"])
wikiCat['title'] = wikiCat['title'].str.replace("https://www.wikihow.com/", "").str.replace("%22", "").str.replace(
    "-", " ").str.lower()
wikiCat['title'] = ["how to " + i for i in wikiCat['title'].tolist()]
wikiCat['category'] = wikiCat['category'].str.lower()

youth


In [169]:
all_categories

['arts and entertainment',
 'cars & other vehicles',
 'computers and electronics',
 'education and communications',
 'family life',
 'finance and business',
 'food and entertaining',
 'health',
 'hobbies and crafts',
 'holidays and traditions',
 'home and garden',
 'personal care and style',
 'pets and animals',
 'philosophy and religion',
 'relationships',
 'sports and fitness',
 'travel',
 'work world',
 'youth']

In [177]:
cat = "sports and fitness"

In [178]:
tasks = wikiCat[wikiCat.category.str.contains(cat)].title.tolist()

# Uniformly Sample negative queries from any task

In [156]:
for cat in all_categories:
    tasks, queries = [], []
    cat = cat.replace(" ", "_")
    print(cat)
    folder = "csv/%s/" % cat
    files = []
    for file in os.listdir(folder):
        if os.stat(folder + file).st_size > 0:
            if ".csv" in file:
                task_name = file.split(".csv")[0].replace("_", " ")
                tmp = pd.read_csv("csv/%s/%s" %(cat,file), names=["q", "qq"], sep=";", header=0)
                try:
                    query = tmp.q.str.lower().str.replace("?", "").unique().tolist() + tmp.qq.str.lower().str.replace("?", "").unique().tolist()
                except:
#                   no sub queries
                    query = tmp.q.str.lower().str.replace("?", "").unique().tolist()
                queries.extend(query)
                tasks.extend([task_name]*len(query))
                files.append(task_name)
    print(len(files))
    break
            
#     #   generate query-task pair dataset
#     x_task, x_query, label = [], [], []
#     for t, pq, nq in zip(tasks, queries, queries[::-1]):
#         x_query.append(pq)
#         x_query.append(nq)
#         x_task.extend([t, t])
#         label.extend([1,0])
#     _df = pd.DataFrame.from_dict({'task': x_task, "query": x_query, "label": label})
#     _df.to_csv("data/query_task_prediction/%s.csv" % cat, index=False, header=False)

youth
4018


# Sample negative queries within the same task

In [ ]:
def read_scraper_file(df):
    try:
        query = df.q.str.lower().str.replace("?", "").unique().tolist() + df.qq.str.lower().str.replace("?", "").unique().tolist()
    except:
#       no sub queries
        query = df.q.str.lower().str.replace("?", "").unique().tolist()
    return query

for cat in all_categories:
    tasks = wikiCat[wikiCat.category.str.contains(cat)].title.tolist()
    sdf = df[df.title.isin(tasks)].drop_duplicates(["sectionLabel"])
    subtasks = df[df.title.isin(tasks)].drop_duplicates(["sectionLabel"]).sectionLabel.unique().tolist()

    cat = cat.replace(" ", "_")
    print(cat)
    folder = "csv/%s/" % cat
    scraper_subtasks = []
    for file in os.listdir(folder):
        if os.stat(folder + file).st_size > 0:
            if ".csv" in file:
                task_name = file.split(".csv")[0].replace("_", " ")
#                 print(task_name)
                if task_name in subtasks:
                    scraper_subtasks.append(task_name)
            
    x_task, x_query, label = [], [], []
    for  name, row in sdf.groupby("title"):
        check = set.intersection(set(row.sectionLabel.tolist()), set(scraper_subtasks))
        if len(check) > 1:
            check = list(check)
            for i in range(len(check)):
                subtask_name = check[i]
                _df = pd.read_csv("csv/%s/%s.csv" %(cat,subtask_name.replace(" ", "_")), names=["q", "qq"], sep=";", header=0)
                pos_query = read_scraper_file(_df)

                neg_query_pool = []
                for _subtask_name in check[:i] + check[i+1:]:
                    _df = pd.read_csv("csv/%s/%s.csv" %(cat, _subtask_name.replace(" ", "_")), names=["q", "qq"], sep=";", header=0)
                    neg_query_pool.extend(read_scraper_file(_df))
                
                if len(pos_query) <= len(neg_query_pool):
                    neg_query = random.sample(neg_query_pool, len(pos_query))
                else:
                    neg_query = []
                    while len(neg_query) < len(pos_query):
                        
                        neg_query.append(random.sample(neg_query_pool, 1))
#                     print(len(neg_query_pool), len(pos_query))
#                     neg_query = neg_query_pool + random.sample(neg_query_pool, len(pos_query) - len(neg_query_pool))
                
                for pq, nq in zip(pos_query, neg_query):
                    x_query.append(pq)
                    x_query.append(nq)
                    x_task.extend([subtask_name, subtask_name])
                    label.extend([1,0])
    _df = pd.DataFrame.from_dict({'task': x_task, "query": x_query, "label": label})
    _df.to_csv("data/query_task_prediction/same_task/%s.csv" % cat, index=False, header=False)
            




arts_and_entertainment
cars_&_other_vehicles
computers_and_electronics
education_and_communications
family_life
finance_and_business
